In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2024-08-18 21:50:44.076982: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-18 21:50:44.994458: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-18 21:50:46.962990: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-18 21:50:50.207605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [3]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    train_df, test_df = data_loader.create_train_test_df(True, True, True)
    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    print(best_estimator.get_all_performance_metrics())
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
    }
    print(output)
    results.append(output)

Number of experiments to run:  1
Numerical Rating
1    2980
3     620
Name: count, dtype: int64
Numerical Rating
1    778
3    134
Name: count, dtype: int64
cluster  predict  veracity
-1       False     1          1381
                   3           296
         True     -1           523
 0       False     3            11
                   1             1
                              ... 
 86      False     3             1
         True     -1            11
 87      False     1            10
                   3             1
         True     -1             1
Name: count, Length: 195, dtype: int64
Number of clusters: 89
Predicted: 1
Confidences: [[0.71251342 0.28748658]]
Predicted (Non Weighted): 3.0
Predicted: 1
Confidences: [[0.64290714 0.35709286]]
Predicted (Non Weighted): 3.0
Predicted: 3
Confidences: [[0.32403794 0.67596206]]
Predicted (Non Weighted): 1.0
Predicted: 1
Confidences: [[0.6250389 0.3749611]]
Predicted (Non Weighted): 3.0
Predicted: 3
Confidences: [[0.46132863 0.53

In [4]:
results_df = pd.json_normalize(results)
results_df

,percentage,score,accuracy,was_supervised_umap_used,metrics.accuracy,metrics.accuracy_not_including_fours,metrics.percentage_of_fours,metrics.percentage_of_no_clusters_in_ground_truth,metrics.precision_on_three,metrics.recall_on_three,metrics.average_confidence_for_3,metrics.precision_on_three_excluding_fours,metrics.recall_on_three_excluding_fours,metrics.cluster_df
0,0.75,1.708899,0.413943,False,0.413943,0.962025,0.569717,0.0,0.862745,0.328358,0,0.862745,0.846154,...


In [5]:
cluster_dfs[0]

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy
0,"In recent months, EPA has finalized strengthen...",3,True,4,"[11.128315925598145, 6.039376735687256, 0.9027...",-1,1677,2200,0.762273
1,"The restriction has stymied efforts to ""fast-t...",1,True,4,"[11.114140510559082, 6.444507598876953, 1.1032...",-1,1677,2200,0.762273
2,The bottom panel of following graphic shows wh...,1,True,1,"[10.96535873413086, 6.5726399421691895, 1.1335...",38,21,21,1.000000
3,", I suggested that there was no evidence that ...",1,True,1,"[10.750276565551758, 6.557883262634277, 1.1452...",16,51,51,1.000000
4,The International Business Times (IBT) reports...,1,True,4,"[11.028009414672852, 6.432281970977783, 0.9894...",-1,1677,2200,0.762273
...,...,...,...,...,...,...,...,...,...
3797,As temperatures have warmed in the United Stat...,3,False,3,"[10.854663848876953, 6.532639503479004, 1.0934...",-1,1677,2200,0.762273
3798,"Today, the US Environmental Protection Agency ...",3,False,3,"[11.164238929748535, 5.918270587921143, 0.9443...",-1,1677,2200,0.762273
3799,"In 2023, sources in both the Cross-State Air P...",3,False,3,"[11.098334312438965, 6.064420700073242, 0.8096...",-1,1677,2200,0.762273
3800,"Some communities, including Boston, Houston, a...",3,False,3,"[11.002217292785645, 6.418937683105469, 1.1341...",23,34,35,0.971429


In [6]:
viz = Visualizer()
df_with_two_dimens = viz.fit_transform(cluster_dfs[0], 'embeddings')
df_with_two_dimens

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy,reduced_embeddings
0,"In recent months, EPA has finalized strengthen...",3,True,4,"[11.128315925598145, 6.039376735687256, 0.9027...",-1,1677,2200,0.762273,"[-1.7573197, 6.2236238]"
1,"The restriction has stymied efforts to ""fast-t...",1,True,4,"[11.114140510559082, 6.444507598876953, 1.1032...",-1,1677,2200,0.762273,"[-5.401451, 3.7327745]"
2,The bottom panel of following graphic shows wh...,1,True,1,"[10.96535873413086, 6.5726399421691895, 1.1335...",38,21,21,1.000000,"[7.880359, -5.414883]"
3,", I suggested that there was no evidence that ...",1,True,1,"[10.750276565551758, 6.557883262634277, 1.1452...",16,51,51,1.000000,"[11.951994, 11.237508]"
4,The International Business Times (IBT) reports...,1,True,4,"[11.028009414672852, 6.432281970977783, 0.9894...",-1,1677,2200,0.762273,"[-5.768486, 4.1529245]"
...,...,...,...,...,...,...,...,...,...,...
3797,As temperatures have warmed in the United Stat...,3,False,3,"[10.854663848876953, 6.532639503479004, 1.0934...",-1,1677,2200,0.762273,"[7.7985497, -0.76715446]"
3798,"Today, the US Environmental Protection Agency ...",3,False,3,"[11.164238929748535, 5.918270587921143, 0.9443...",-1,1677,2200,0.762273,"[-2.151344, 6.882872]"
3799,"In 2023, sources in both the Cross-State Air P...",3,False,3,"[11.098334312438965, 6.064420700073242, 0.8096...",-1,1677,2200,0.762273,"[-1.2368455, 4.708243]"
3800,"Some communities, including Boston, Houston, a...",3,False,3,"[11.002217292785645, 6.418937683105469, 1.1341...",23,34,35,0.971429,"[5.885763, -2.7862296]"
